In [1]:
import random
import copy
import time
import math
import nglview

In [11]:
#sigma = float(input('Enter inter-atomic distance: '))
#natoms = int(input('Enter number of atoms: '))
natoms = 3

mass = [0.0]*natoms
#m = input('Enter mass of atoms ')
for n in range(natoms):
    mass[n] = 39.5
    
coord = [[0.0 for tmp in range(3)] for n in range(natoms)]
for n in range(natoms):
    coord[n][0] = random.uniform(-4.0, 4.0)
    coord[n][1] = random.uniform(-4.0, 4.0)
    coord[n][2] = random.uniform(-4.0, 4.0)
    
vel = [[0.0 for tmp in range(3)] for n in range(natoms)]
for n in range(natoms):
    vel[n][0] = random.gauss(0,0.01)
    vel[n][1] = random.gauss(0,0.01)
    vel[n][2] = random.gauss(0,0.01)

$$r=[(x_2-x_1)^2 + (y_2-y_1)^2 + (z_2-z_1)^2]^{\frac{1}{2}}$$

In [12]:
def distance(atom1,atom2):
    x = atom2[0]-atom1[0]
    y = atom2[1]-atom1[1]
    z = atom2[2]-atom1[2]
    
    rr = x*x + y*y + z*z
    return math.sqrt(rr)

$$V_{ij}(r) = 4\epsilon \left[(\frac{\sigma}{r_{ij}})^{12}-(\frac{\sigma}{r_{ij}})^6 \right]$$

In [13]:
def lennard_jones(rr,sigma=0.3345,eps=0.0661):
    
    return 4*eps*(sigma**12/rr**12-sigma**6/rr**6)

$$T = \frac{1}{2}\sum_i^N m_iv_i^2$$

In [14]:
def kinetic_energy(natoms,mass,vel):
    
    sum = 0.0
    for n in range(natoms):
        vv = vel[n][0]**2 + vel[n][1]**2 + vel[n][2]**2
        sum+=mass[n]*vv
    return 0.5*sum


$$V=\sum_i\sum_{j\neq i}V_{ij}$$

In [15]:
def potential_energy(natoms,coord):
    pot=0
    for n1 in range(natoms):
        for n2 in range(n1+1,natoms):
            r=distance(coord[n1],coord[n2])
            pot+=lennard_jones(r) 
    
    for n1 in range(natoms):
        r=distance(coord[n1],[0,0,0])
        if r>10:
    #        pass
            pot+=0.5*00.1*(r-4)**2
    return pot

$$E = T + V$$

In [16]:
def energy(natoms,coord,mass,velocity):
    pot=potential_energy(natoms,coord)
    kin=kinetic_energy(natoms,mass,velocity)
    
    return kin+pot

In [17]:
nrg = energy(natoms,coord,mass,vel)
print("potential energy: %f"%(potential_energy(natoms,coord)))
print("kinetic energy: %f"%(kinetic_energy(natoms,mass,vel)))
print("total energy: %f"%(energy(natoms,coord,mass,vel)))

potential energy: -0.000000
kinetic energy: 0.024010
total energy: 0.024010


In [18]:
print(nrg)

0.024010327162375632


$$\frac{\partial V}{\partial x} \approx \frac{V(x+\Delta x) - V(x-\Delta x)}{2\Delta x}  $$

In [19]:
def finite_difference(natoms,coord):
    dx = [[0.0 for tmp in range(3)] for n in range(natoms)]
    
    for n in range(natoms):
        for l in range(3):
            f=[0.0]*2
            for i,x in enumerate((1,-1)):
                crd = copy.deepcopy(coord)
                crd[n][l]+=x*0.0001
                
                f[i]=potential_energy(natoms,crd)

            dx[n][l]=(f[0]-f[1])/0.0002

    return dx                

$${\bf a} = -\frac{1}{\bf m}\frac{\partial V}{\partial {\bf x}}$$

In [20]:
def acceleration(natoms,coord,mass):
    a = [[0.0 for tmp in range(3)] for n in range(natoms)]
    
    dx = finite_difference(natoms,coord)
    
    for n in range(natoms):
        for l in range(3):
            a[n][l]=-1*dx[n][l]/mass[n]
            
    return a



$${\bf r}^{n+1} = {\bf r}^{n} + {\bf v}^{n}\Delta t + {\bf a}^{n}\frac{{\Delta t}^2}{2}$$
$${\bf v}^{n+1} = {\bf v}^{n} + \frac{1}{2}({\bf a}^{n} + {\bf a}^{n+1})\Delta t $$

In [21]:
def verlet(natoms,c0,v0,mass,ts):
    
    c1 = [[0.0 for tmp in range(3)] for n in range(natoms)]
    v1 = [[0.0 for tmp in range(3)] for n in range(natoms)]
    
    a0 = acceleration(natoms,c0,mass)
    
    for n in range(natoms):
        for l in range(3):
            c1[n][l]=c0[n][l]+v0[n][l]*ts+a0[n][l]*ts**2/2
    

    a1 = acceleration(natoms,c1,mass)
    
    for n in range(natoms):
        for l in range(3):
            v1[n][l]=v0[n][l]+0.5*(a0[n][l]+a1[n][l])*ts
    
    return c1,v1

In [22]:
def temperature(natoms,v):
    avg=0
    for i in range(natoms):
        avg+=math.sqrt(v[i][0]**2+v[i][1]**2+v[i][2]**2)
    avg/=natoms
    
    return avg*avg*math.pi*mass[0]/1000/8/8.314*1000

In [23]:
def print_xyz(natoms,c,v,energy,step,temp,fname="traj.xyz"):
    fout=open("traj.xyz","a")
    fout.write("%d\n"%(natoms))
    fout.write("Step %4d E: %f K: %f\n"%(step,energy,temp))
    for n in range(natoms):
        fout.write("Ar %10.7f %10.7f %10.7f %10.7f %10.7f %10.7f\n"%(c[n][0],c[n][1],c[n][2],\
                                                                   v[n][0],v[n][1],v[n][2]))
    fout.close()
    return
        

In [24]:
c=coord
v=vel
fname="traj.xyz"
open(fname, 'w').close()
for step in range(100000):
    
    nrg=energy(natoms,c,mass,v)
    temp=temperature(natoms,v)
    
    print_xyz(natoms,c,v,nrg,step,temp)
    #print(v[1][0])
    
    c,v=verlet(natoms,c,v,mass,0.1)
    #print(v)

KeyboardInterrupt: 

In [27]:
from nglview.datafiles import PDB, XTC